#Initial Setup

In [27]:
!pip install evaluate # HF evaluate lib for metrics
!pip install bitsandbytes # 8 bit Adam optimizer
!pip install trl # Transformer Reinforcement Learning


In [28]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple, Optional, Any, Union
import json
import random
from dataclasses import dataclass
from collections import defaultdict
import subprocess
import sys
from pydantic import BaseModel, Field, field_validator, ConfigDict
from typing import Optional
import evaluate # HF evaluate lib for metrics
import wandb

try:

  from transformers import(
      AutoModelForCausalLM, # pretrained models
      AutoTokenizer,
      AutoModelForSequenceClassification, # For reward models
      Trainer,  # HF training loop
      TrainingArguments,  # Training hyperparams
      DataCollatorForLanguageModeling, # BAtch collation for LM trainning
      BitsAndBytesConfig, # For 8bit quantization
      )

  from datasets import Dataset as HFDataset, load_dataset
  from peft import(
      LoraConfig,
      get_peft_model, # convert model to peft model
      TaskType, # task type enum
      PeftModel, # peft model class
      prepare_model_for_kbit_training # prepare quantized model for training
  )

  from accelerate import Accelerator # Distributed training support
  from accelerate.utils import set_seed # seed setting for accelerate
  import bitsandbytes as nbn # 8bit adam optimizer

  HF_AVAILABLE = True
  print("HuggingFace libraries available with enhancements")

except ImportError:

  HF_AVAILABLE = False
  print("HuggingFace libraries not available - install with: pip install transformers datasets peft accelerate bitsandbytes")



try:

  from trl import (
      SFTTrainer, # supervised fine tuning trainer
      DPOTrainer, # direct preference optimization trainer
      RewardTrainer, # reward model trainer
      DataCollatorForCompletionOnlyLM # Collator for instruction tuning
  )
  TRL_AVAILABLE = True
  print("TRL (Transformer Reinforcement Learning) available")

except ImportError:

  TRL_AVAILABLE = False
  print("TRL (Transformer Reinforcement Learning) not available")



# Init wandb for experiment tracking
USE_WANDB = False

if USE_WANDB:
  wandb.init(
      project='llm-alignment-textbook',
      config={ # Hyperparams
               'achitecture': "GPT-2",
               'dataset':"custom-instructions",
               'learning_rate':5e-5,
      },
      model='online' # offline for local logging instead
  )



import warnings
warnings.filterwarnings('ignore')

# Seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
  torch.cuda.manual_seed_all(42) # CUDA seeds for GPU operations

# Device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print(f"PyTorch version: {torch.__version__}")



HuggingFace libraries available with enhancements
TRL (Transformer Reinforcement Learning) not available
Using device: cpu
PyTorch version: 2.8.0+cu126
